In [27]:
# coding: utf-8

import pandas as pd
from impala.dbapi import connect
from impala.util import as_pandas
import ast
import math

def agg_montly_total_amount_by_product_cate(year):
    
    conn = connect(host='salest-master-server', port=21050)
    cur = conn.cursor()

    cur.execute('USE salest')
    cur.execute(
    """
        SELECT SUBSTR(view_tr_receipt.date_receipt_num,1,7) AS year_month, 
              view_tr_receipt.num_of_product, view_tr_receipt.sales_amount AS total_amount,
            ext_menumap_info.product_name, ext_menumap_info.cate_name, ext_menumap_info.price
        FROM (SELECT * FROM ext_tr_receipt WHERE SUBSTR(date_receipt_num,1,4) = '""" + year + "'" +
    """) view_tr_receipt JOIN ext_menumap_info USING (product_code)"""
    )

    df_tr_receipt_menumap = as_pandas(cur)
    conn.close()
    
    def aggregation(row):
        total_amount = row['total_amount'].sum()
        product_names = row['product_name'].sum()
        return pd.Series([total_amount,product_names], index=['total_amount','product_names'])
    
    df_monthly_product_tr = df_tr_receipt_menumap.groupby(['year_month','cate_name']).apply(aggregation)
    
    df_default = genDefaultMontlyCateTotalAmountDataFrame(df_monthly_product_tr,year, 'cate_name')
    df_all_monatly_sales_volume = pd.merge(df_default, df_monthly_product_tr, left_index=True, right_index=True, how='outer').fillna(0).sort_index(ascending='1')

    return df_default

    def post_aggregation(row):
        return row['total_amount_x'] + row['total_amount_y']
    
    df_all_monatly_sales_volume['total_amount'] = df_all_monatly_sales_volume.apply(post_aggregation, axis=1)
    df_all_monatly_sales_volume.drop(['total_amount_x','total_amount_y'], axis=1, inplace=True)

    def gen_dict_total_amount(month_rows):
        monthlyDict = {}
        monthlyDictKey = month_rows.index.get_level_values('year_month')[0]
        
        monthCateItemsStr = "{"
        for item in zip(month_rows.index.get_level_values('cate_name'),month_rows['total_amount']):
            monthCateItemsStr += "'{0}':{1},".format(item[0],item[1]);
        
        monthCateItemsStr = monthCateItemsStr[:-1]
        monthCateItemsStr += "}"
        
        monthlyDict = ast.literal_eval(monthCateItemsStr)
        monthlyDict['year_month'] = month_rows.index.get_level_values('year_month')[0]

        return monthlyDict
    
    return df_all_monatly_sales_volume

    mothlyTotalAmountDictItems = df_all_monatly_sales_volume.groupby(df_all_monatly_sales_volume.index.get_level_values('year_month')).apply(gen_dict_total_amount)    

    mothlyTotalAmountDict = {}
    mothlyTotalAmountList = []
    for item in mothlyTotalAmountDictItems:
        mothlyTotalAmountList.append(item)
    mothlyTotalAmountDict['total_amount'] = mothlyTotalAmountList

    return mothlyTotalAmountDict


#################################################################################################################################
# Utility Functions
#################################################################################################################################

def gen_dict_total_amount(row):
    
    monthlyDict = {}
    monthlyDictStr = "{"
    for key,value in zip(row.index, row): 
        monthlyDictStr += "'{0}':{1},".format(key,value)
    
    monthlyDictStr = monthlyDictStr[:-1]
    monthlyDictStr += "}"
    
    monthlyDict = ast.literal_eval(monthlyDictStr)
    monthlyDict['year_month'] = row.name
 
    return monthlyDict


def genDefaultMontlyCateTotalAmountDataFrame(df_monthly_product_tr,year, second_index_name):
    unique_count_of_category = len(df_monthly_product_tr.index.get_level_values(1).unique())

    month_index_arr = []
    cate_index_arr = []

    for month in range(1,13):
        for cate in range(unique_count_of_category):
            month_index_arr.append("{0}-{1:02d}".format(year,month))

    unique_cate_index_arr = df_monthly_product_tr.index.get_level_values(1).unique()
    #df_monthly_product_tr.ix[0:unique_count_of_category].index.get_level_values(1)
    
    for item in unique_cate_index_arr:
        print ">>" + item
        
    print unique_cate_index_arr
    
    for month in range(1,13):
        for cate in unique_cate_index_arr:
            cate_index_arr.append(cate)

    full_month_cate_multi_index = pd.MultiIndex.from_tuples(zip(month_index_arr, cate_index_arr), 
                                                  names=['year_month', second_index_name])
    df_full_month_cate_default = pd.DataFrame(0, index=full_month_cate_multi_index, columns=['total_amount'])
    return df_full_month_cate_default

##########################################################################################################################3

agg_montly_total_amount_by_product_cate('2015')

>>더치
>>라떼
>>버블티
>>브레드
>>생과일주스
>>아이스티.에이드
>>요거트.스무디
>>젤라또아이스크림
>>차 허브티
>>커피
>>프라페
>>﻿커피
>>원두
>>빙수
['\xeb\x8d\x94\xec\xb9\x98' '\xeb\x9d\xbc\xeb\x96\xbc'
 '\xeb\xb2\x84\xeb\xb8\x94\xed\x8b\xb0'
 '\xeb\xb8\x8c\xeb\xa0\x88\xeb\x93\x9c'
 '\xec\x83\x9d\xea\xb3\xbc\xec\x9d\xbc\xec\xa3\xbc\xec\x8a\xa4'
 '\xec\x95\x84\xec\x9d\xb4\xec\x8a\xa4\xed\x8b\xb0.\xec\x97\x90\xec\x9d\xb4\xeb\x93\x9c'
 '\xec\x9a\x94\xea\xb1\xb0\xed\x8a\xb8.\xec\x8a\xa4\xeb\xac\xb4\xeb\x94\x94'
 '\xec\xa0\xa4\xeb\x9d\xbc\xeb\x98\x90\xec\x95\x84\xec\x9d\xb4\xec\x8a\xa4\xed\x81\xac\xeb\xa6\xbc'
 '\xec\xb0\xa8 \xed\x97\x88\xeb\xb8\x8c\xed\x8b\xb0'
 '\xec\xbb\xa4\xed\x94\xbc' '\xed\x94\x84\xeb\x9d\xbc\xed\x8e\x98'
 '\xef\xbb\xbf\xec\xbb\xa4\xed\x94\xbc' '\xec\x9b\x90\xeb\x91\x90'
 '\xeb\xb9\x99\xec\x88\x98']


total_amount
year_month cate_name              
2015-01    더치                    0
           라떼                    0
           버블티                   0
           브레드                   0
           생과일주스                 0
           아이스티.에이드              0
           요거트.스무디               0
           젤라또아이스크림              0
           차 허브티                 0
           커피                    0
           프라페                   0
           ﻿커피                   0
           원두                    0
           빙수                    0
2015-02    더치                    0
           라떼                    0
           버블티                   0
           브레드                   0
           생과일주스                 0
           아이스티.에이드              0
           요거트.스무디               0
           젤라또아이스크림              0
           차 허브티                 0
           커피                    0
           프라페                   0
           ﻿커피                   0
           원두                    0
           빙수                    0
2015-03    더치                    0
           라떼                    0
...                            ...
2015-10    원두                    0
           빙수                    0
2015-11    더치                    0
           라떼                    0
           버블티                   0
           브레드                   0
           생과일주스                 0
           아이스티.에이드              0
           요거트.스무디               0
           젤라또아이스크림              0
           차 허브티                 0
           커피                    0
           프라페                   0
           ﻿커피                   0
           원두                    0
           빙수                    0
2015-12    더치                    0
           라떼                    0
           버블티                   0
           브레드                   0
           생과일주스                 0
           아이스티.에이드              0
           요거트.스무디               0
           젤라또아이스크림              0
           차 허브티                 0
           커피                    0
           프라페                   0
           ﻿커피                   0
           원두                    0
           빙수                    0

[168 rows x 1 columns]